# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#drop missing values
df_cleaned = spaceship.dropna()
df_cleaned.shape

(6606, 14)

In [4]:
#format cabin values
df_cleaned['Cabin'] = df_cleaned['Cabin'].str[0]
df_cleaned['Cabin'].value_counts()

/var/folders/76/0pntsdld7bg_p5gzgms4gghw0000gn/T/ipykernel_85484/3361844794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Cabin'] = df_cleaned['Cabin'].str[0]


Cabin
F    2152
G    1973
E     683
B     628
C     587
D     374
A     207
T       2
Name: count, dtype: int64

In [5]:
#Drop PassengerId and Name
df_cleaned2 = df_cleaned.drop(columns=['PassengerId','Name'])

In [6]:
#feature encoding.
# One-hot encoding for sex and title
non_numerical_cols = df_cleaned2.select_dtypes(include=['object', 'category']).columns
df_cleaned3 = pd.get_dummies(df_cleaned2, columns=non_numerical_cols)

df_cleaned3.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False


In [7]:
# Drop the 'Transported' column
features = df_cleaned3.drop(columns=['Transported'])

# Display the first few rows of the resulting DataFrame
features.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,True,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,True,...,False,False,True,False,False,False,False,True,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,True,...,False,False,False,False,False,False,False,True,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,True,...,False,False,False,False,False,False,False,True,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,True,...,False,False,True,False,False,False,False,True,True,False


In [8]:
# Select the 'Transported' column
target = df_cleaned3[['Transported']]

# Display the first few rows of the 'target' DataFrame
target.head()

,Transported
0,False
1,True
2,False
3,False
4,True


In [9]:
#Perform Train Test Split
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [10]:
#create an instance of the normalizer
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [11]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [12]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [13]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.632911,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.227848,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.189873,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.658228,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.784810,0.0,0.054775,0.0,0.07774,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [14]:
# Check the number of samples in X_train_norm
n_samples = X_train_norm.shape[0]
print(f"Number of samples in training data: {n_samples}")

Number of samples in training data: 5284


In [20]:
#Pasting regressor with  max_samples=1000,
# Initialize the Pasting model
pasting_reg = BaggingRegressor(
  estimator=DecisionTreeRegressor(max_depth=20),
  n_estimators=100,
  max_samples=1000,
  bootstrap=False  # This ensures that no bootstrap is done, which is characteristic of Pasting
)

# Train the model with normalized data
pasting_reg.fit(X_train_norm, y_train)

# Evaluate the model's performance
pred = pasting_reg.predict(X_test_norm)

print("MAE:", mean_absolute_error(pred, y_test))
print("RMSE:", mean_squared_error(pred, y_test, squared=False))
print("R2 score:", pasting_reg.score(X_test_norm, y_test))

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


MAE: 0.27499939593675904
RMSE: 0.3772845612555009
R2 score: 0.4306254393529767


- Evaluate your model

In [21]:
# Evaluate the model's performance
pred = pasting_reg.predict(X_test_norm)

print("MAE:", mean_absolute_error(pred, y_test))
print("RMSE:", mean_squared_error(pred, y_test, squared=False))
print("R2 score:", pasting_reg.score(X_test_norm, y_test))

MAE: 0.27499939593675904
RMSE: 0.3772845612555009
R2 score: 0.4306254393529767


### Grid Search

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.
- Run Grid Search
- Evaluate your model

In [23]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Define the parameter grid with the correct parameter names for BaggingRegressor
param_grid = {
    'base_estimator__max_depth': [None, 10, 20, 30],  # Depth for Decision Tree
    'n_estimators': [50, 100, 200]                    # Number of base estimators
}

# Set up Grid Search with DecisionTreeRegressor as the base estimator
grid_search = GridSearchCV(BaggingRegressor(base_estimator=DecisionTreeRegressor()), 
                           param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error')

# Assuming X_train and y_train are already defined
grid_search.fit(X_train, y_train)

# Best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Calculate RMSE from the best score
best_rmse = np.sqrt(-grid_search.best_score_)

# Output best parameters and best RMSE
print("Best Parameters:", best_params)
print("Best RMSE from Grid Search:", best_rmse)

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/py

Best Parameters: {'base_estimator__max_depth': 10, 'n_estimators': 200}
Best RMSE from Grid Search: 0.37323105437110876


In [25]:
#Checking with optimal Parameters: base_estimator__max_depth': 10, 'n_estimators': 200
pasting_reg = BaggingRegressor(
  estimator=DecisionTreeRegressor(max_depth=10),
  n_estimators=200,
  max_samples=1000,
  bootstrap=False  # This ensures that no bootstrap is done, which is characteristic of Pasting
)

# Train the model with normalized data
pasting_reg.fit(X_train_norm, y_train)

# Evaluate the model's performance
pred = pasting_reg.predict(X_test_norm)

print("MAE:", mean_absolute_error(pred, y_test))
print("RMSE:", mean_squared_error(pred, y_test, squared=False))
print("R2 score:", pasting_reg.score(X_test_norm, y_test))

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


MAE: 0.2765834930081046
RMSE: 0.3755357737244801
R2 score: 0.4358915306126243


### Compare results before and after Grid Search 

#### Parameters: max_depth : 20, n_estimators: 100
- MAE: 0.27499939593675904
- RMSE: 0.3772845612555009
- R2 score: 0.4306254393529767

#### Parameters: max_depth': 10, 'n_estimators': 200:

- MAE: 0.2766127377227113
- RMSE: 0.3745605087462347
- R2 score: 0.43881770115104735
